# Geração de instâncias

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from udgp import M1, M2, M4, M5, Instance

In [2]:
env = gp.Env()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br


In [5]:
coords = np.array(
    [
        [-0.9523699364, 0.0159052548, -0.0840802250],
        [-0.2528949024, 0.8598484003, -0.3332183372],
        [-0.3357005052, -0.8500181306, 0.2812584200],
        [0.7960710440, 0.5155096551, -0.1218613860],
        [0.7448954577, -0.5412454414, 0.2579017563],
        [-0.0442130155, 0.1953980103, 0.5377653956],
        [0.0442118576, -0.1953977484, -0.5377656238],
    ]
)

In [9]:
N = 6
instance = Instance.random(N, freq=True)
# instance = Instance.from_coords(coords, freq=True)

instance.view_input()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
instance.dist

array([1.5  , 2.45 , 2.871, 3.465, 3.775, 4.242, 4.5  ], dtype=float16)

In [6]:
instance.reset()
m = M4(instance, n=5, env=env)
m.setParam("TimeLimit", 60)
m.optimize(log=True)

m.instance.view()


Set parameter LogToConsole to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license - for non-commercial use only - registered to braun@cos.ufrj.br
Optimize a model with 435 rows, 206 columns and 1234 nonzeros
Model fingerprint: 0x23dccb3b
Model has 15 quadratic constraints
Variable types: 146 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e+00]
  RHS range        [7e-01, 1e+01]
Presolve removed 80 rows and 19 columns
Presolve time: 0.00s
Presolved: 550 rows, 247 columns, 1662 nonzeros
Presolved model has 60 bilinear constraint(s)
Variable types: 187 continuous, 60 integer (60 binary)

Root relaxation: objective 0.000000e+00, 181 iterations, 0.00 seconds (

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
print(instance.coords)


[[-0.854 -0.182 -0.753]
 [-0.419 -0.043 -1.756]
 [-0.062  0.623 -0.96 ]
 [ 0.     0.     0.   ]
 [ 0.176 -0.498 -0.96 ]
 [ 0.646  0.195 -1.677]]


In [10]:
# TODO: evitar que o cara encontre a mesma solução errada várias vezes.

previous_a = []

while not instance.is_solved():
    # while instance.fixed_n < instance.n:
    print("reset")
    instance.reset()
    # instance.reset_random_core(5)

    m = M4(instance, n=4, env=env)
    # m.Params.SolutionLimit = 1
    m.optimize()

    previous_a.append(list(m.a_ijk_values()))
    print(list(m.a_ijk_values()))

    while instance.fixed_n < instance.n:
        print("step")
        m = M4(instance, n=1, env=env)
        m.setParam("TimeLimit", 60)
        m.optimize()

        if m.Status is not GRB.OPTIMAL:
            break

instance.view()

reset


KeyboardInterrupt: 

In [11]:
instance.reset()
previous_a = []

m = M4(instance, n=4, previous_a=previous_a, env=env)
m.optimize()

previous_a.append(list(m.a_ijk_values()))

instance.view()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [17]:
instance.reset()
previous_a = []

m = M4(instance, n=4, previous_a=previous_a, env=env)
m.optimize()

previous_a.append(list(m.a_ijk_values()))

instance.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [18]:
instance.reset()
previous_a = []

m = M4(instance, n=4, previous_a=previous_a, env=env)
m.optimize()

previous_a.append(list(m.a_ijk_values()))

instance.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
instance.reset()
# previous_a = []

m = M4(instance, n=4, previous_a=previous_a, env=env)
m.optimize()

previous_a.append(list(m.a_ijk_values()))

instance.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol